### **MonthlyMean rainfall in Zugspitze/Germany**

In [71]:
#Import earth engine
import ee

#Trigger the authentication flow
ee.Authenticate()

#Initialize the library
ee.Initialize(project='modis-410817')

In [26]:
# Import geemap
import geemap as emap

In [76]:
# Create a region of interest (ROI)
#roi = ee.Geometry.Rectangle([10.97, 47.40, 11.47, 47.50])
roi = ee.Geometry.Rectangle([10.956, 47.381, 11.213, 47.470])

# Define the image collection
era5land = ee.ImageCollection("ECMWF/ERA5/DAILY") \
    .select('total_precipitation') \
    .filter(ee.Filter.calendarRange(2000, 2020, 'year')) \
    .filter(ee.Filter.Or(
        ee.Filter.calendarRange(1, 3, 'month'),
        ee.Filter.calendarRange(10, 12, 'month')
    )) \
    .filterBounds(roi) \
    .map(lambda img: img.set({'month': ee.Date(ee.Number(img.get('system:time_start'))).get('month'),
                               'year': ee.Date(ee.Number(img.get('system:time_start'))).get('year')}))

In [77]:
# Multiply precipitation values by 1000
era5P = era5land.map(lambda image: image.multiply(1000)
                                         .set('year', image.get('year'))
                                         .set('month', image.get('month'))
                                         .set('system:time_start', image.get('system:time_start')))

In [78]:
# Define the list of months and years
months = [1, 2, 3, 10, 11, 12]
years = list(range(2000, 2001))

byYearMonth = ee.ImageCollection.fromImages(
    [era5P.filter(ee.Filter.calendarRange(y, y, 'year'))
             .filter(ee.Filter.calendarRange(m, m, 'month'))
             #.select('total_precipitation')
             .mean()
             .set('year', y)
             .set('month', m)
             .set('system:time_start', ee.Date.fromYMD(y, m, 1))
     for y in years for m in months]
)

In [69]:
# Get the image IDs
image_ids = byYearMonth.aggregate_array('system:index').getInfo()
print('Total images', len(image_ids))

Total images 6


In [33]:
# Get a list of images
image_list = byYearMonth.toList(byYearMonth.size())

#Clip for displaying in map
img = ee.Image(image_list.get(0)).clip(roi)

In [34]:
# Visualization palette for total precipitation
visTp = {
  'min': 0.0,
  'max': 500,
  'palette': ['ffffff', '00ffff', '0080ff', 'da00ff', 'ffa400', 'ff0000']
}

In [ ]:
#Create a center the map object
Map = emap.Map()
Map.center_object(roi, zoom=12)
# Add the layers to the map object.
Map.addLayer(img,visTp,'Rainfall')

# Draw the rectangle on the map
Map.addLayer(roi, {'color': 'red'}, 'roi')

#Add a layer control panel to the map
Map.addLayerControl()

#Display the map
Map

In [79]:
# Export loop
exportArgs = {
    'folder': "RainfallMean_Zugspitze",
    'region': roi,
    'scale': 500,
    'fileFormat': "GeoTIFF",
    'crs': 'EPSG:4326',
    'maxPixels': 1e10,
    'formatOptions': {
        'cloudOptimized': True
    }
}

# Convert the image collection into a list
imgList = byYearMonth.toList(byYearMonth.size())
numImg = imgList.size().getInfo()

for i in range(numImg):
    # Get the images from the list
    image = ee.Image(imgList.get(i))

    # Extract month and year information from image properties
    month = image.get('month')
    year = image.get('year')

    # Get the date of the image
    #date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')

    exportArgs['image'] = image.select('total_precipitation')
    exportArgs['description'] = 'RainfallMean_Zugspitze_{}_{}_{}'.format(year.getInfo(), month.getInfo(), i + 1)
    exportArgs['fileNamePrefix'] = '{}_{}'.format(year.getInfo(), month.getInfo())

    # Export the images
    task = ee.batch.Export.image.toDrive(**exportArgs)
    task.start()
    print('Started task: ', i + 1)

Started task:  1
Started task:  2
Started task:  3
Started task:  4
Started task:  5
Started task:  6
